# lib & load_data

In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# !pip install konlpy

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import reuters
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt, Kkma
okt = Okt()
kkma = Kkma()
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from hanspell import spell_checker

In [6]:
df_train = pd.read_csv("./data/text/train_data.csv")
df_test = pd.read_csv("./data/text/test_data.csv")


# 전처리

## 정규 표현식으로 치환
분, 주문번호 변환

In [7]:
# 정규표현식 함수 정의

def re_sub(df):
    convert_ls =[]
    for idx in (df['text']):
        idx = re.sub("\d\d\d\d"," @",idx)
        idx = re.sub("\d\d분"," #분",idx)
        
        ## 띄어쓰기, 맞춤법
        spelled_sent = spell_checker.check(idx)
        hanspell_sent = spelled_sent.checked
        
        convert_ls.append(hanspell_sent)

    convert_txt = pd.Series(convert_ls, name = 'convert_ls')
    df = pd.concat([df,convert_txt],axis = 1)

    return df

In [8]:
df_train = re_sub(df_train)
df_test = re_sub(df_test)

display(df_train.tail())
display(df_test.tail())

,text,intent,label,convert_ls
120,배달끝났어,배달완료,6,배달 끝났어
121,음식배달완료,배달완료,6,음식 배달 완료
122,음식배달완료했어,배달완료,6,음식 배달 완료했어
123,음식배달끝냈어,배달완료,6,음식 배달 끝냈어
124,음식배달끝났어,배달완료,6,음식 배달 끝났어


,text,intent,label,convert_ls
9,영수증 9893,영수증번호,4,영수증 @
10,10분후 도착,소요시간선택,5,#분 후 도착
11,20분후 도착,소요시간선택,5,#분 후 도착
12,배달끝,배달완료,6,배달 끝
13,배달완료,배달완료,6,배달 완료


### train_data label 확인tail

In [9]:
# label
pd.Series.unique(df_train['intent'])

array(['운행시작', '가게전화', '가게도착', '픽업완료', '영수증번호', '소요시간선택', '배달완료'],
      dtype=object)

## 형태소로 분리 , df에 열로 추가

### tokenizer 테스트

In [ ]:
# okt_ls = []
# for i in range(len(text_data)):
#     okt_text = okt.pos(text_data['convert_ls'][i])
#     okt_ls = okt_ls + okt_text

# pd.unique(okt_ls)

In [ ]:
# kkma_ls = []
# for i in range(len(text_data)):
#     kkma_text = kkma.pos(text_data['convert_ls'][i])
#     kkma_ls = kkma_ls + kkma_text

# pd.unique(kkma_ls)

### kkma가 더 적합해보임   ----> okt로 적힌 함수명 고치기

토큰화 / 불용어 제거 / token_len_max

In [10]:
# kkma 토큰화 함수정의  

def kkma_tokenizer(input_df):
    # 일단 제거할 품사집합을 설정(품사들만 설정)
    valid_pos = ['NNG','VV','SW','MAG'] # 'NNG=보통명사','VV=동사','SW=기타기호','MAG=일반부사'

    # token_text 먼저 선언
    input_df['token_text'] = np.nan
    
    # 토큰화하기 위한 컬럼 설정
    token_ls =[] # 나중에 서버에 넣을 때는 필요없음 / 인공지능 학습용 데이터 만들기
    for i in range(len(input_df)):
        
        # tokenize
        token_text = kkma.pos(input_df['convert_ls'][i])
        
        # 불용어 제거
        ls = [] # 해당 열의 토큰 텍스트에 해당되는 값
        for token in token_text:
            
            if token[1] in valid_pos: # token은 튜플 값 / ('단어':'품사 형태')
                ls.append(token[0])
                token_ls.append(token[0])
            input_df['token_text'][i] = ls

In [11]:
kkma_tokenizer(df_train)
display(df_train)
kkma_tokenizer(df_test)
display(df_test)

C:\Users\parkgun\anaconda3\envs\pydatavenv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\parkgun\anaconda3\envs\pydatavenv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,text,intent,label,convert_ls,token_text
0,운행시작해,운행시작,0,운행 시작해,"[운행, 시작하]"
1,운행시작하자,운행시작,0,운행 시작하자,"[운행, 시작]"
2,운행하자,운행시작,0,운행하자,[운행]
3,운행해,운행시작,0,운행해,[운행]
4,운행시작해주세요,운행시작,0,운행 시작해주세요,"[운행, 시작하]"
...,...,...,...,...,...
120,배달끝났어,배달완료,6,배달 끝났어,"[배달, 끝나]"
121,음식배달완료,배달완료,6,음식 배달 완료,"[음식, 배달, 완료]"
122,음식배달완료했어,배달완료,6,음식 배달 완료했어,"[음식, 배달, 완료]"
123,음식배달끝냈어,배달완료,6,음식 배달 끝냈어,"[음식, 배달, 끝내]"


C:\Users\parkgun\anaconda3\envs\pydatavenv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\parkgun\anaconda3\envs\pydatavenv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,text,intent,label,convert_ls,token_text
0,운행시작해,운행시작,0,운행 시작해,"[운행, 시작하]"
1,배달 시작할게,운행시작,0,배달 시작할게,"[배달, 시작하]"
2,가게에 전화,가게전화,1,가게에 전화,"[가게, 전화]"
3,식당에 전화해,가게전화,1,식당에 전화해,"[식당, 전화]"
4,식당 왔어,가게도착,2,식당 왔어,"[식당, 오]"
5,가게 도착,가게도착,2,가게 도착,"[가게, 도착]"
6,픽업완료,픽업완료,3,픽업 완료,"[픽업, 완료]"
7,픽업했어,픽업완료,3,픽업했어,[픽업]
8,주문번호 7289,영수증번호,4,주문번호 @,"[주문, 번호, @]"
9,영수증 9893,영수증번호,4,영수증 @,"[영수증, @]"


## 정수 인코딩 
위 코드에서 token_ls 라는 변수명 사용 ==> 바꿔서 변수명이 서로 겹치지 않도록 유지

In [75]:
# 토크나이저 최적화
token_ls1 = [] # 위의 토큰화 함수 변수와 동일 / 바꿔줘야 할 필요가 있음

for i in range(len(df_train)):
    token_ls1 = token_ls1 + df_train['token_text'][i]

token_ls1 = pd.Series(token_ls1)

# tokenizer fit
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token_ls1)
tokenizer.fit_on_

vocab_size = len(pd.unique(token_ls1))

display(token_ls1)

token_ls1.to_csv("./data/text/token_ls.csv")

0       운행
1      시작하
2       운행
3       시작
4       운행
      ... 
283     배달
284     끝내
285     음식
286     배달
287     끝나
Length: 288, dtype: object

In [4]:
?Tokenizer().fit_on_texts()

Object `Tokenizer().fit_on_texts()` not found.


# 정수인코딩 컬럼 생성
위의 코드와 같이 token_ls가 반복 // 이를 구분할 수 있는 아규먼트로 설정하자

In [58]:
## df의 정수인코딩 column 생성 및 반영 함수화

# 두번째 인수에서 헷갈릴 수도 있음 / 변수명을 변화시킬지 생각해보기


def int_encode(df, token_ls):

    tokenizer.fit_on_texts(token_ls)

    df['integer_encode'] = np.nan
    
    for i in range(len(df)):
        
        # integer encode
        seq = tokenizer.texts_to_sequences(df['token_text'])

        df['integer_encode'] = seq

In [59]:
int_encode(df_train,token_ls = token_ls)
int_encode(df_test,token_ls = token_ls)

display(df_train.head())
display(df_test.tail())

,text,intent,label,convert_ls,token_text,integer_encode
0,운행시작해,운행시작,0,운행 시작해,"[운행, 시작하]","[13, 15]"
1,운행시작하자,운행시작,0,운행 시작하자,"[운행, 시작]","[13, 18]"
2,운행하자,운행시작,0,운행하자,[운행],[13]
3,운행해,운행시작,0,운행해,[운행],[13]
4,운행시작해주세요,운행시작,0,운행 시작해주세요,"[운행, 시작하]","[13, 15]"


,text,intent,label,convert_ls,token_text,integer_encode
0,운행시작해,운행시작,0,운행 시작해,"[운행, 시작하]","[13, 15]"
1,배달 시작할게,운행시작,0,배달 시작할게,"[배달, 시작하]","[2, 15]"
2,가게에 전화,가게전화,1,가게에 전화,"[가게, 전화]","[5, 8]"
3,식당에 전화해,가게전화,1,식당에 전화해,"[식당, 전화]","[4, 8]"
4,식당 왔어,가게도착,2,식당 왔어,"[식당, 오]","[4, 17]"


## 패딩

In [75]:
df_train = df_train[['integer_encode','label']]
df_test = df_test[['integer_encode','label']]


In [ ]:
max_len = 8

train_padded = pad_sequences(df_train['integer_encode'], maxlen=max_len)
test_padded = pad_sequences(df_test['integer_encode'], maxlen=max_len)

# train_test_split

In [78]:
x_data_train,x_data_valid, y_data_train, y_data_valid = \
train_test_split(train_padded,
                 df_train['label'],
                 test_size=0.3,
                 random_state=0,
                 shuffle = True)

In [ ]:
# x_data_train,x_data_valid, y_data_train, y_data_valid = \
# train_test_split(df_train.drop('label', axis=1, inplace=False),
#                  train_data['label'],
#                  test_size=0.3,
#                  random_state=0,
#                  shuffle = True)

# Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

## label one_hot_encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

y_data_train = to_categorical(y_data_train) # 훈련용 레이블의 원-핫 인코딩
y_data_valid = to_categorical(y_data_valid) # valid용 레이블의 원-핫 인코딩

##call_back

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 학습 시 정답률이 높아지면 멈추는 것 / 과적합
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 성능이 가장 좋을 때 저장

## modeling

In [81]:
intent = pd.unique(train_data['label'])
print(intent)
intent_count = intent.shape[0]
intent_count

[0 1 2 3 4 5 6]


7

In [ ]:
model = Sequential() # 모델을 시작하기 위한 시작함수 (모델은 층을 쌓는다?)
model.add(Embedding(vocab_size, 120)) # add 는 층 추가
model.add(LSTM(120)) # LSTM은 순환신경망 모델 // 장기간으로 기억할 수 있도록 구성
model.add(Dense(intent_count, activation='softmax')) # 다양한 값이 나온다? 0-1 사이 출력

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc']) # 모델 구성 작업

In [ ]:
history = model.fit(x_data_train,
                    y_data_train,
                    batch_size = 10, # 한번에 학습시키는 데이터 양
                    epochs=10,       # 몇 번 학습시킬 지 --> 과적합 위험
                    callbacks=[es, mc], # 피드백? 학습 피드백
                    validation_data=(x_data_valid, # 평가용 데이터 // 모의고사 데이터
                                     y_data_valid))

# predict

In [ ]:
pred = model.predict(test_padded) # 테스트 데이터 인풋
y_pred = [np.argmax(i) for i in pred] # softmax값 중 가장 큰 것을 불러와서 인덱스에 해당하는 열에 입력

0     0
1     0
2     1
3     1
4     2
5     2
6     3
7     3
8     4
9     4
10    5
11    5
12    0
13    6
dtype: int64

In [95]:
pd.concat([test_data,pd.Series(y_pred,name= 'prediction')],axis=1)

,padded,label,prediction
0,"[0, 0, 0, 0, 0, 0, 13, 15]",0,0
1,"[0, 0, 0, 0, 0, 0, 3, 15]",0,0
2,"[0, 0, 0, 0, 0, 0, 5, 7]",1,1
3,"[0, 0, 0, 0, 0, 0, 4, 7]",1,1
4,"[0, 0, 0, 0, 0, 0, 4, 17]",2,2
5,"[0, 0, 0, 0, 0, 0, 5, 1]",2,2
6,"[0, 0, 0, 0, 0, 0, 14, 9]",3,3
7,"[0, 0, 0, 0, 0, 0, 0, 14]",3,3
8,"[0, 0, 0, 0, 0, 0, 11, 2]",4,4
9,"[0, 0, 0, 0, 0, 0, 0, 10]",4,4


In [ ]:
import sklearn
print(sklearn.metrics.classification_report(test_data['label'],y_pred))